In [44]:
import pandas as pd
import numpy as np

In [45]:
english_movies=pd.read_csv("till 2023.csv")
hindi_movies=pd.read_excel("hindi_movies.xlsx")

In [46]:
english_movies.head()

movie_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                                 tag      id  
0  cchpounder joeldavidmoore wesstudi jamescamero...   19995  
1  johnnydepp orlandobloom jackdavenport goreverb...     285  
2  christophwaltz rorykinnear stephaniesigman sam...  206647  
3  tomhardy christianbale josephgordon-levitt chr...   49026  
4  darylsabara samanthamorton pollywalker andrews...   49529

In [47]:
hindi_movies.head()

id               movie_title  \
0  872906                     Jawan   
1  554600  Uri: The Surgical Strike   
2  781732                    Animal   
3  975533               The Archies   
4  734253                 Adipurush   

                                                 tag  
0  shahrukhkhan nayanthara vijaysethupathi atlee ...  
1  vickykaushal pareshrawal yamigautam adityadhar...  
2  ranbirkapoor rashmikamandanna anilkapoor sande...  
3  agastyananda suhanakhan khushikapoor zoyaakhta...  
4  prabhas saifalikhan kritisanon omraut action a...

In [48]:
all_movies = pd.concat([hindi_movies, english_movies], axis=0)
all_movies

id               movie_title  \
0      872906                     Jawan   
1      554600  Uri: The Surgical Strike   
2      781732                    Animal   
3      975533               The Archies   
4      734253                 Adipurush   
...       ...                       ...   
6856   818397                    Memory   
6857   558915          The Color Purple   
6858   823452      The Boys in the Boat   
6859   365620                   Ferrari   
6860  1036619                Good Grief   

                                                    tag  
0     shahrukhkhan nayanthara vijaysethupathi atlee ...  
1     vickykaushal pareshrawal yamigautam adityadhar...  
2     ranbirkapoor rashmikamandanna anilkapoor sande...  
3     agastyananda suhanakhan khushikapoor zoyaakhta...  
4     prabhas saifalikhan kritisanon omraut action a...  
...                                                 ...  
6856  jessicachastain petersarsgaard merrittwever mi...  
6857  fantasiabarrino tarajip.henson daniellebrooks ...  
6858  callumturner joeledgerton peterguinness george...  
6859  adamdriver penelopecruz shailenewoodley michae...  
6860  danlevy ruthnegga himeshpatel danlevy comedy d...  

[11174 rows x 3 columns]

In [49]:
all_movies.drop_duplicates(subset ="movie_title", keep = 'last', inplace = True)
all_movies = all_movies.sample(frac=1).reset_index(drop=True)

In [50]:
all_movies

id             movie_title  \
0      809929    Aquarium of the Dead   
1       25462  Two Can Play That Game   
2      410554              First Kill   
3      199408                    1971   
4      590995       The Craft: Legacy   
...       ...                     ...   
10838  671626                    Ouch   
10839  770254     Back to the Outback   
10840  365222                Ip Man 3   
10841  247645                   Queen   
10842  408272                Catfight   

                                                     tag  
0      vivicaa.fox evaceja d.c.douglas glennmiller ho...  
1      mo'nique vivicaa.fox tamalajones markbrown com...  
2      brucewillis haydenchristensen gethinanthony st...  
3      manojbajpayee ravikishan sunilchavan amritsaga...  
4      caileespaeny gideonadlon loviesimone zoelister...  
...                                                  ...  
10838  manojbajpayee poojachopra neerajpandey drama c...  
10839  islafisher timminchin ericbana clareknight fam...  
10840  miketyson lynnhung kwok-kwanchan wilsonyip act...  
10841  kanganaranaut rajkummarrao lisahaydon anuragka...  
10842  sandraoh anneheche aliciasilverstone onurtukel...  

[10843 rows x 3 columns]

In [51]:
all_movies.isnull().sum()

id             0
movie_title    0
tag            0
dtype: int64

In [52]:
all_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10843 entries, 0 to 10842
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           10843 non-null  int64 
 1   movie_title  10843 non-null  object
 2   tag          10843 non-null  object
dtypes: int64(1), object(2)
memory usage: 254.3+ KB


In [53]:
from sklearn.feature_extraction.text import CountVectorizer

In [54]:
cv=CountVectorizer()

In [55]:
vectors=cv.fit_transform(all_movies["tag"]).toarray()

In [56]:
vectors.shape

(10843, 17543)

In [57]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [58]:
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
similarity=cosine_similarity(vectors)

In [60]:
similarity.shape

(10843, 10843)

In [61]:
similarity[0]

array([1.        , 0.375     , 0.13363062, ..., 0.11785113, 0.14433757,
       0.14433757])

In [62]:
def recommend(movie):
    index = all_movies[all_movies['movie_title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        movie
        print(i)
        print(all_movies.iloc[i[0]].movie_title)
        

In [63]:
recommend("Avatar")

(2889, 0.5892556509887895)
Aliens
(9519, 0.5892556509887895)
Avatar: The Way of Water
(643, 0.5555555555555555)
The League of Extraordinary Gentlemen
(1202, 0.5555555555555555)
Justice League
(2342, 0.5555555555555555)
Highlander: Endgame


In [64]:
import pickle

In [65]:
pickle.dump(all_movies.to_dict(),open("movies_dict.pkl","wb"))

In [66]:
pickle.dump(similarity,open("similarity.pkl","wb"))

In [67]:
import pickle
import pandas as pd
import gradio as gr
import requests
import numpy as np

def fetch_poster(movie_id):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key=bd9d93e4649f8d7b42731beda117f8c6&language=en-US"
    data = requests.get(url).json()

    if 'poster_path' in data and data['poster_path']:
        poster_path = data['poster_path']
        full_path = f"https://image.tmdb.org/t/p/w500/{poster_path}"
        return full_path
    else:
        return np.NaN

def recommend(movie):
    index = movies[movies['movie_title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    recommended_movie_names = []
    recommended_movie_posters = []

    for i in distances[1:11]:
        movie_id = movies.iloc[i[0]].id
        poster = fetch_poster(movie_id)

        if not pd.isna(poster):
            recommended_movie_posters.append(poster)
            recommended_movie_names.append(movies.iloc[i[0]].movie_title)

    return recommended_movie_names[:5], recommended_movie_posters[:5]

# Load the data
movies = pickle.load(open('movies_dict.pkl','rb'))
movies = pd.DataFrame(movies)
similarity = pickle.load(open('similarity.pkl','rb'))

# Define the function to be called when the button is pressed
def get_recommendations(selected_movie):
    recommended_movie_names, recommended_movie_posters = recommend(selected_movie)
    return recommended_movie_names, recommended_movie_posters

# Convert movie_list to a regular list
movie_list = movies['movie_title'].tolist()

# Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown('# Movie Recommender System')
    selected_movie = gr.Dropdown(choices=movie_list, label="Type or select a movie from the dropdown")
    recommend_button = gr.Button("Show Recommendation")
    
    # Create columns for movie recommendations
    with gr.Row() as row:
        output_boxes = []
        for i in range(5):
            with gr.Column() as col:
                output_text = gr.Textbox(label="Movie Name")
                output_image = gr.Image(label=f"Movie Poster {i+1}", height=500, width=500)  # Set size here
                output_boxes.append((output_text, output_image))
    
    def show_recommendations(selected_movie):
        recommended_movie_names, recommended_movie_posters = get_recommendations(selected_movie)
        outputs = []
        for name, poster in zip(recommended_movie_names, recommended_movie_posters):
            outputs.append(name)
            outputs.append(poster)
        return outputs
    
    recommend_button.click(fn=show_recommendations, inputs=selected_movie, outputs=[box for pair in output_boxes for box in pair])

# Launch the interface
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://2df3c0ccd9ab3486a0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
